stages of file generatation and usage

1. initial file by opening, creating empty data sets for markers and channels, storing meta data (file id, anything else?)
       - def h5_init(bam_name)

2. Write marker info and channels to file -- hopefully auto chunking will be fast and effecient. Assuming it's not too slow, I think writing all loci to file is best, filtering out what sites to use can be decided in subsequent steps. lot a memory potentially.


3. construct a common set of loci by inserting empty channels as needed. will need to make choices about filtering at the same stage. this seems like the most challening part (pho king A)



unaddressed challenges

- need to modify marker names to account for inserts -- perhaps change them to be format: chrom ipos? as in "A i10"?

- how to effeciently insert empty channels where individuals don't match --- sheesh. this chunk of code may help:

penta_ex = make_penta(seq_str, qual_str, depth_str, "T", qual_min=0)
print(penta_ex)
penta_ex = np.insert(penta_ex, obj = [0, 0], values = -999, axis=0)
penta_ex.shape




In [2]:
import h5py
import numpy as np
import os

os.chdir("/home/silastittes/Dropbox/penta_channel/")


In [36]:
def init_penta(bam_in, h5_out):

    with h5py.File(h5_out, 'w') as f:

        channels = f.create_dataset('channels', (0, 0), maxshape=(None,None), dtype='f8', chunks=True)
        makers = f.create_dataset('markers', (0, 0), maxshape=(None,2), dtype = 'S100', chunks=True)

        metadata = {'id': bam_in
                   }
        f.attrs.update(metadata)
        
def add_pentas(h5_out, penta_arr, marker_arr):
    try:
        with h5py.File(h5_out, 'a') as f:
            channels = f['channels']
            ch_end = channels.shape[0]
            
            shp = channels.shape[0] + penta_arr.shape[0]
            channels.resize((shp, 5))    
            channels[ch_end:shp] = penta_arr 
            
            markers = f['markers']
            mrk_end = markers.shape[0]
            mrk_shp = markers.shape[0] + marker_arr.shape[0]
            markers.resize((mrk_shp, 2))
            markers[mrk_end:mrk_shp] = marker_arr
            
    except FileNotFoundError:
        print("{0} not found".format(h5_out))

        
#older version        
def make_penta(seq_str, qual_str, depth_str, ref, qual_min = 50):
    seq_str = seq_str.upper()
    nuc_dict = {"A":0, "T":1, "G":2, "C":3, "*":4}
    nucs = {"A": 0, "T": 0, "G":0, "C":0, "*":0}

    if depth_str == "0" and seq_str == "*" and qual_str == "*":
        seq_channel = [[0,0,0,0,0]]
        #return np.array(seq_channel)
        return seq_channel

    else:
        i = 0
        q = 0
        inserts = list()
        gaps = list()

        while len(seq_str) > i:
            if seq_str[i] == "$": i += 1
            if seq_str[i] == "^": i += 2
            if seq_str[i] in [".", ","]:
                if ord(qual_str[q]) > qual_min:
                    nucs[ref] += 1
                q += 1

            if seq_str[i] in nucs:
                if ord(qual_str[q]) > qual_min:
                    nucs[seq_str[i]] += 1
                q += 1

            if seq_str[i] in ["+"]:
                i += 1
                j = 0
                insert_str = ""
                while seq_str[i].isnumeric():
                    insert_str += seq_str[i]
                    i += 1
                    j += 1
                insert_int = int(insert_str)
                insert_seq = seq_str[i:i + insert_int]
                while len(inserts) < insert_int:
                    inserts.append([0,0,0,0,0])

                for s in range(len(insert_seq)):
                    inserts[s][nuc_dict[insert_seq[s]]] +=1

                i += len(insert_str) + insert_int - 2

            if seq_str[i] in ["-"]:
                i += 1
                j = 0
                gap_str = ""
                while seq_str[i].isnumeric():
                    gap_str += seq_str[i]
                    i += 1
                    j += 1
                gap_int = int(gap_str)

                i += len(gap_str) + gap_int - 2
            i += 1

        seq_channel = list(nucs.values())
        inserts.insert(0,seq_channel)
        #return np.array(inserts)
        return inserts

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    #IUFHIUHWIUFHWDF
    
    
    
    

#newer version SEEMS TO BE WORKING, BUT MORE TESTS NEEDED!!!
def make_penta(seq_str, qual_str, depth_str, ref, qual_min = 50):
    seq_str = seq_str.upper()
    
    nuc_dict = {"A":0, "T":1, "G":2, "C":3, "*":4, "N":5}
    nucs = {"A": 0, "T": 0, "G":0, "C":0, "*":0, "N":0}
    inserts = {"A": 0, "T": 0, "G":0, "C":0, "*":0, "N":0}
    
    if depth_str == "0" and seq_str == "*" and qual_str == "*":
        seq_channel = [[0,0,0,0,0], [0,0,0,0,0]]
        return seq_channel
    
    else:
        i = 0
        q = 0
        while i < len(seq_str):
            if seq_str[i] == "$": 
                i += 1
            elif seq_str[i] == "^": 
                i += 2
                
            elif seq_str[i] in [".", ","]:
                if ord(qual_str[q]) > qual_min:
                    nucs[ref] += 1
                i += 1
                q += 1

            elif seq_str[i] in nucs:
                if ord(qual_str[q]) > qual_min:
                    nucs[seq_str[i]] += 1
                i += 1
                q += 1

                
            elif seq_str[i] == "+":
                i += 1
                j = 0
                insert_str = ""
                while seq_str[i].isnumeric():
                    insert_str += seq_str[i]
                    i += 1
                    j += 1
                insert_int = int(insert_str)
                insert_seq = seq_str[i:i + insert_int]
                i += insert_int

                for s in range(len(insert_seq)):
                    inserts[insert_seq[s]] += 1

            elif seq_str[i] == "-":
                i += 1
                j = 0
                gap_str = ""
                while seq_str[i].isnumeric():
                    gap_str += seq_str[i]
                    i += 1
                    j += 1
                gap_int = int(gap_str)
                i += gap_int
                
        seq_channel = [list(nucs.values())[0:5], list(inserts.values())[0:5]]
        #seq_channel = [list(nucs.values()), list(inserts.values())]
        return seq_channel



In [4]:
init_penta("data/bam/no.bam", "data/h5/test.h5")

In [37]:
ref = "A"
qual_min = 0



#seq_str = "*$.$.$.$.$.$.+5AGACT.+5AGACT.+5AGACT.+5AGACT.+5AGACT,+5agact.+5AGACT.+5AGACT.+5AGACT.+5AGACT.+5AGACT.+5AGACT"
#qual_str = "221102222222222222"
#depth_str = len(qual_str)

#seq_str = "*"
#qual_str = "*"
#depth_str = "0"

seq_str = ",.$.....,*.,.,...,,,.,..^+."
qual_str = "<<<22<<<<<<<<<<<2<;<172#"
depth_str = len(qual_str)

seq_str = "*+3AGG**a*G**+3AGG+4TTTT-5gccttcA"
seq_str = "***a*G+3TTT**cA"
qual_str = "222222222%"
depth_str = len(qual_str)

seq_str = ".G$,+6ACAACC-3CCC-1A^~.+5AAAAA+3AAA-6AAAAAA"
qual_str = "AAAA"
depth_str = "4"


#seq_str = "AA+10ATGC*ATGC*GC"
#qual_str = "2222"
#depth_str = "0"


make_penta(seq_str, qual_str, depth_str, ref = "N", qual_min=0)


[[0, 0, 1, 0, 0], [11, 0, 0, 3, 0]]

In [6]:
with h5py.File("src/test.h5", 'a') as f:
    channels = f['channels']
    ch = channels[:]  
    markers = f['markers']
    mrk = markers[:]  
print(ch, "\n", mrk)

with h5py.File("src/test2.h5", 'a') as f:
    channels = f['channels']
    ch2 = channels[:]  
    markers = f['markers']
    mrk = markers[:]  
print(ch2, "\n", mrk)

np.mean(ch == ch2)
print(ch.shape, ch2.shape)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 
 [[b'reference' b'1']
 [b'reference' b'i1']
 [b'reference' b'2']
 ...
 [b'reference' b'i4999']
 [b'reference' b'5000']
 [b'reference' b'i5000']]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]] 
 [[b'reference' b'i1']
 [b'reference' b'i2']
 [b'reference' b'i3']
 ...
 [b'reference' b'i4966']
 [b'reference' b'i4967']
 [b'reference' b'i4968']]
(10000, 5) (4968, 5)


/home/silastittes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


In [7]:
c1 = make_penta(seq_str, qual_str, depth_str, ref, qual_min=0)
chrom = "ch"
pos = "1"
mrk_init = [[chrom, pos]]
mrk_init.append([chrom, pos])
np.array([[chrom, "i"+str(int(pos)+ i)] for i in range(len(c1))], dtype = "S100")
c1

ll = []
ll.append(1)
ll

[1]

In [8]:
def fasta_dict(file_name = "seqs.fa"):
    seq_dict = dict()
    with open(file_name) as fa:
        for ln in fa:
            line = ln.strip()
            if line and line[0] == ">":
                seq_name = line[1:]
                if seq_name not in seq_dict:
                    seq_dict[seq_name] = ""
                else:
                    raise ValueError("Fasta headers are not unique.")
            else:
                seq_dict[seq_name] += ln.strip()
    return seq_dict

def make_fastas(file_name = 'seqs.fa'):    
    fastas = fasta_dict("notebooks/seqs.fa")
    for name, seq in fastas.items():
        with open('seq_{0}.fa'.format(name), 'w+') as out_file:
            out_file.write(">reference_{0}\n{1}\n".format(name, seq))
        
#make_fastas("notebooks/seqs.fa")

make_fastas("notebooks/seqs.fa")

FileNotFoundError: [Errno 2] No such file or directory: 'notebooks/seqs.fa'

In [9]:
import numpy as np

arr_empty = np.zeros((100,2))
np.count_nonzero(arr_empty, axis = 0)

array([0, 0])